<center>
# Text Mining 2018

## Assignment 2
</center><div class="pull-right">
김제경 20176005 
이희랑 20186014  
</div>


The purpose of assignment2 is to derive and implement PLSA(Probabilistic Latent Semantic Analysis) algorithm.  
In this assignment, we will compare two kinds of approachs;  
** 1) One is the model without Background Language Model, **  
** 2) The other is the model with Background Language Model **

The reason why we implement two approach is we want to compare the result and performance of two methods, which are to remove stopwords directly in preprocessing step and to define stopwords by using background language model.

#  
## 1. Preprocessing

Preprocessing for documents is same as previous assignment, so we put preprocessing code in the end of the report.   
Now we have only 10 documents, but we plan to gather more data for our term project purpose.

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
path = 'E:\\unist\\2-1\\'
df = pd.read_csv(path + 'abstract_preprocessed.csv', encoding='cp949')
df.head()

FileNotFoundError: File b'E:\\unist\\2-1\\abstracts2.csv' does not exist

#####  
### 1.1. Make TDM using sklearn package
we defined 2 corpus with stopwords and without stopwords to compare performance of Background Language Model(probablistic stopwords define) with method which is to remove stopwords directly in preprocessing step.

In [3]:
# corpus without stopwords
vect = CountVectorizer(stop_words='english')
corpus1 = vect.fit_transform(df['all'].tolist()).toarray()

# corpus with stopwords
vect2 = CountVectorizer(stop_words=None, min_df=0, max_df=100000)
corpus2 = vect2.fit_transform(df['all'].tolist()).toarray()

In [4]:
print(corpus1.shape)
print(corpus2.shape)

(10, 484)
(10, 577)


In [5]:
print(corpus2)

[[0 0 0 ..., 0 0 1]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 2 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


** small additional preprocessing **   
We removed the words appeared only once because we consider those
are meaningless. 

In [7]:
corpus_noback = corpus1[:, np.sum(corpus1, axis=0)>1]
corpus_back = corpus2[:, np.sum(corpus2, axis=0)>1]

print(corpus_noback.shape)
print(corpus_back.shape)

(10, 196)
(10, 249)


In [8]:
words_noback = [a for a,b in zip(sorted(list(vect.vocabulary_.keys())), np.sum(corpus1, axis=0)>1) if b ]
words_back = [a for a,b in zip(sorted(list(vect2.vocabulary_.keys())), np.sum(corpus2, axis=0)>1) if b ]

In [9]:
stopwords_index = [i for i, a in enumerate(words_back) if a not in words_noback]
lamb = np.sum([a for i, a in enumerate(corpus_back) if i in stopwords_index]) / np.sum(corpus_back)
lamb

0.10221205186880244

-> By comparing shape of corpus in above, we can assume that there are 56 stopwords in corpus.  
-> In this manner, we will set λb = (total count of stopwords / total count of words) ~ 0.10 later. 

#  
## 2. PLSA Implementation

In [10]:
# function for compute normalize
def normalize(vec):
    s = sum(vec)
    for i in range(len(vec)):
        vec[i] = vec[i] * 1.0 / s   
    return vec

In [13]:
def PLSA(tdm, number_of_documents, number_of_vocabs, number_of_topics, lamb, number_of_iteration):
    
    # Initialize parameter by random
    pi = normalize(np.random.random(size = (number_of_documents, number_of_topics)))
    topic_word_prob = normalize(np.random.random(size = (number_of_topics, number_of_vocabs)))  
    background_word_prob = np.sum(tdm, axis=0) / np.sum(tdm)

    # Initialize topic probability and background probability 
    topic_prob = np.zeros([number_of_documents, number_of_vocabs, number_of_topics], dtype=np.float)
    background_prob = np.zeros([number_of_documents, number_of_vocabs], dtype=np.float)

    iteration = number_of_iteration
    ll = []
    for i in range(iteration):

        print('%i\'s Iteration' % (i))
        print('E-step')
        for d_i, document in enumerate(tdm):
            for w_i in range(number_of_vocabs):
                prob = pi[d_i, :] * topic_word_prob[:, w_i]
                topic_prob[d_i][w_i] = normalize(prob)

                background_prob[d_i][w_i] = (lamb * background_word_prob[w_i]) / ((lamb * background_word_prob[w_i]) + ((1-lamb) * np.sum(pi[d_i, :] * topic_word_prob[:, w_i])))

        print('M-step')
        for d_i in range(number_of_documents):
            for j in range(number_of_topics):
                s = 0
                for w_i in range(number_of_vocabs):
                    count = tdm[d_i][w_i]
                    s += count * (1-background_prob[d_i][w_i]) * topic_prob[d_i, w_i, j]
                pi[d_i][j] = s
            pi[d_i] = normalize(pi[d_i])

        for j in range(number_of_topics):
            for w_i in range(number_of_vocabs):
                s = 0
                for d_i in range(number_of_documents):
                    count = tdm[d_i][w_i]
                    s += count * (1-background_prob[d_i][w_i]) * topic_prob[d_i, w_i, j]
                topic_word_prob[j][w_i] = s
            topic_word_prob[j] = normalize(topic_word_prob[j])

        # log-likelihood
        print('Calculate Log-Likelihood')
        likelihood = 0
        for d_i in range(number_of_documents):
            for w_i in range(number_of_vocabs):
                likelihood += tdm[d_i][w_i] * np.log((lamb * background_word_prob[w_i]) + ((1-lamb) *  np.sum(pi[d_i, :] * topic_word_prob[:, w_i])))
        print(likelihood, '\n')
        ll.append(likelihood)
    
    return pi, topic_word_prob, ll, background_word_prob

####  
### 2.1. Compute Log-Likelihood in 2 ways
#### 1. Model without Background Language Model
 - Using corpus without stopwords (preprocessed by nltk's stopwords)
 - As don't use backgoround language model, set λb = 0
 
#### 2. Model applying Background Language Model
 - Using corpus with stopwords
 - λb = (total count of stopwords / total count of words) ~ 0.1
 
We chose the number of topics to 2, because we gather our data in two category; NLP, Robotics
  
-------------------------------------------------------------------------------- 
  
*** We set random seed to '1' because we need to compare the PLSA result with same initialization point

In [16]:
iteration = 100
'''
print('1. Stopword를 임의로 제거(패키지의 stopword 이용)해준 Corpus 사용 : λb = 0')
np.random.seed(1)
pi_noback, topic_word_prob_noback, likelihood_noback = PLSA(corpus_noback, corpus_noback.shape[0], corpus_noback.shape[1], 2, 0, iteration)
'''

print('\n\n\n2. 전체 단어를 포함한 Corpus 사용(Background Language Model)')
np.random.seed(1)
pi_back, topic_word_prob_back, likelihood_back, back = PLSA(corpus_back, corpus_back.shape[0], corpus_back.shape[1], 2, lamb, iteration)




2. 전체 단어를 포함한 Corpus 사용(Background Language Model)
0's Iteration
E-step
M-step
Calculate Log-Likelihood
-6477.11031526 

1's Iteration
E-step
M-step
Calculate Log-Likelihood
-6443.16364739 

2's Iteration
E-step
M-step
Calculate Log-Likelihood
-6407.8318217 

3's Iteration
E-step
M-step
Calculate Log-Likelihood
-6371.41476444 

4's Iteration
E-step
M-step
Calculate Log-Likelihood
-6340.41847902 

5's Iteration
E-step
M-step
Calculate Log-Likelihood
-6317.54595603 

6's Iteration
E-step
M-step
Calculate Log-Likelihood
-6301.56175834 

7's Iteration
E-step
M-step
Calculate Log-Likelihood
-6290.68261507 

8's Iteration
E-step
M-step
Calculate Log-Likelihood
-6283.00872049 

9's Iteration
E-step
M-step
Calculate Log-Likelihood
-6277.332939 

10's Iteration
E-step
M-step
Calculate Log-Likelihood
-6273.2685342 

11's Iteration
E-step
M-step
Calculate Log-Likelihood
-6270.51726236 

12's Iteration
E-step
M-step
Calculate Log-Likelihood
-6268.62605979 

13's Iteration
E-step
M-step
Calculat

In [18]:
print('\nTopic Word Distribution Sorted by Topic 1')
pd.DataFrame(np.round(back.T, 5), columns = ['Topic1'],\
            index = words_back).sort_values('Topic1', ascending=False).head(20)


Topic Word Distribution Sorted by Topic 1


,Topic1
the,0.04958
to,0.04195
of,0.03814
and,0.02822
we,0.02517
in,0.02441
learning,0.02365
that,0.02288
model,0.01831
robot,0.01678


In [12]:
print('Log-likelihood of the PLSA without background language model is', likelihood_noback[-1])
print('Log-likelihood of the PLSA with background language model is', likelihood_back[-1])

Log-likelihood of the PLSA without background language model is -3537.5972008
Log-likelihood of the PLSA with background language model is -6249.21107765


Log-likelihood of the PLSA without background language model is smaller than the other.  
So, we will use PLSA with background language model.

In [13]:
print('First 3 iteration\'s log-likelihood :', likelihood_noback[:3])
print('Last 3 iteration\'s log-likelihood :', likelihood_noback[-3:])

First 3 iteration's log-likelihood : [-3794.7740197260196, -3755.7057788517482, -3713.1587442420996]
Last 3 iteration's log-likelihood : [-3537.5972202549028, -3537.5972110530761, -3537.5972007963865]


#  
## 3. Conclusion
### 3.1. Check P(z|d)

In [14]:
pd.DataFrame(np.round(pi_noback, 5), columns = ['Topic1', 'Topic2'],\
               index=['Doc 1', 'Doc 2', 'Doc 3', 'Doc 4', 'Doc 5', 'Doc 6', 'Doc 7', 'Doc 8', 'Doc 9', 'Doc10'])


,Topic1,Topic2
Doc 1,0.00000,1.00000
Doc 2,0.44050,0.55950
Doc 3,1.00000,0.00000
Doc 4,1.00000,0.00000
Doc 5,1.00000,0.00000
Doc 6,0.00000,1.00000
Doc 7,0.00000,1.00000
Doc 8,0.00000,1.00000
Doc 9,0.94327,0.05673
Doc10,0.81460,0.18540


As a result of PLSA,  
** Topic1 ~ ( Doc 3, Doc 4, Doc 5, Doc 9, Doc 10 )**  
** Topic2 ~ ( Doc 1, Doc 2, Doc 6, Doc 7, Doc 8 ) **

### 3.2. Check P(w|z)

In [15]:
print('\nTopic Word Distribution Sorted by Topic 1')
pd.DataFrame(np.round(topic_word_prob_noback.T, 5), columns = ['Topic1', 'Topic2'],\
            index = words_noback).sort_values('Topic1', ascending=False).head(20)


Topic Word Distribution Sorted by Topic 1


,Topic1,Topic2
model,0.07235,0.00000
network,0.04518,0.00456
style,0.02713,0.00000
sequence,0.02713,0.00000
neural,0.02634,0.00739
generative,0.02412,0.00000
text,0.02412,0.00000
adversarial,0.02110,0.00000
architecture,0.01809,0.00000
language,0.01809,0.00000


Topic Word Distribution says 'model', 'network', 'sequence', 'generative', 'text', 'language' has high probability for topic1.  
We can assume that this topic is related to NLP category.

In [16]:
print('\nTopic Word Distribution Sorted by Topic 2')
pd.DataFrame(np.round(topic_word_prob_noback.T, 5), columns = ['Topic1', 'Topic2'],\
            index = words_noback).sort_values('Topic2', ascending=False).head(20)


Topic Word Distribution Sorted by Topic 2


,Topic1,Topic2
learning,0.01708,0.05741
reinforcement,0.00000,0.04532
robot,0.01332,0.03984
policy,0.00000,0.03626
task,0.01123,0.03008
long,0.00000,0.02493
training,0.01750,0.01857
based,0.00870,0.01838
navigation,0.00000,0.01813
learn,0.00000,0.01813


In [ ]:
print('\nTopic Word Distribution Sorted by Topic 1')
pd.DataFrame(np.round(topic_word_prob_noback.T, 5), columns = ['Topic1', 'Topic2'],\
            index = words_noback).sort_values('Topic1', ascending=False).head(20)

Topic Word Distribution says 'learning', 'reinforcement', 'robot', 'policy', 'navigation' has high probability for topic2.  
We can assume that this topic is related to Robotics category.

#  
#  
#  
#  
# APPENDIX : Text Preprocessing

### Package Import

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
#from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
#from nltk.stem import PorterStemmer
from nltk.tag import pos_tag, pos_tag_sents
from collections import Counter, OrderedDict
#from scipy.spatial.distance import pdist,squareform

import re

from nltk.corpus import stopwords

### Data Loading

In [35]:
path = 'E:\\unist\\2-1\\'
data = pd.read_csv(path+'abstract.csv', encoding='cp949')
data

,title,contents
0,Beyond word importance: using contextual decom...,The driving force behind the recent success of...
1,Generating Wikipedia by Summarizing Long Seque...,We show that generating English Wikipedia arti...
2,MaskGAN: Better Text Generation via Filling in...,Recurrent neural networks (RNNs) are a common ...
3,Natural TTS Synthesis By Conditioning WaveNet ...,"This paper describes Tacotron 2, a neural netw..."
4,SHAPED: Shared-Private Encoder-Decoder for Tex...,Supervised training of abstractive language ge...
5,PRM-RL: Long-range Robotic Navigation Tasks by...,"We present PRM-RL, a hierarchical method for l..."
6,Collective Robot Reinforcement Learning with D...,"In principle, reinforcement learning and polic..."
7,Deep Reinforcement Learning for Robotic Manipu...,Reinforcement learning holds the promise of en...
8,Learning to Manipulate Granular Media with a R...,"In this paper, we examine the problem of robot..."
9,Supervision via Competition: Robot Adversaries...,There has been a recent paradigm shift in robo...


#  
## 1. Do Pre-Processing



1.1. Normalization

### Normalization 1, 2, 3


In [36]:
l1 = ['\(CD\)', 'CD', '\(MOS\)', '\(RNNs\)', 'LSTM', 'Yelp', 'SST', 'WaveNet', 'Q\-function', 'GAN', 'RNN', 'Tacotron\ 2', 'F0', 'FST', 'PRM\-RL', 'PRMs',\
      'PRM', 'RL', '\-\ ', '\-', 'TTS', 'MOS']
l2 = ['', 'contenxtual decomposition', '', '', 'long short term memory', 'corporation', 'project', 'generative neural network',\
     'distribution', 'generative adversarial network', 'recurrent neural network', 'neural network model', 'accent',\
     'finite state transducer', 'probabilistic road map reinforcement learning', 'probabilistic road map','probabilistic road map', \
      'reinforcement learning', '', ' ', 'text to speach', 'mean opinion score']

for a, b in zip(l1, l2):
    data['contents'] = data['contents'].str.replace(a, ' '+b)
    data['title'] = data['title'].str.replace(a, ' '+b)

### Normalization 4


In [37]:
data['title'] = data['title'].str.lower()
data['contents'] = data['contents'].str.lower()

data.head()

,title,contents
0,beyond word importance: using contextual decom...,the driving force behind the recent success of...
1,generating wikipedia by summarizing long seque...,we show that generating english wikipedia arti...
2,mask generative adversarial network: better te...,recurrent neural networks are a common metho...
3,natural text to speach synthesis by condition...,"this paper describes neural network model, a ..."
4,shaped: shared private encoder decoder for t...,supervised training of abstractive language ge...


###  
### Tokenization
Split sentences to words list by using non-word character

1.1. Tokenization:  tokenize each document into tokens



### Tokenize


In [38]:
data['title'].apply(lambda x : word_tokenize(x))

0    [beyond, word, importance, :, using, contextua...
1    [generating, wikipedia, by, summarizing, long,...
2    [mask, generative, adversarial, network, :, be...
3    [natural, text, to, speach, synthesis, by, con...
4    [shaped, :, shared, private, encoder, decoder,...
5    [probabilistic, road, map, reinforcement, lear...
6    [collective, robot, reinforcement, learning, w...
7    [deep, reinforcement, learning, for, robotic, ...
8    [learning, to, manipulate, granular, media, wi...
9    [supervision, via, competition, :, robot, adve...
Name: title, dtype: object

In [39]:
doc = pd.DataFrame()
doc['id'] = data.index
doc['title'] = data['title'].apply(lambda x : word_tokenize(x))
doc['contents'] = data['contents'].apply(lambda x : word_tokenize(x))
doc.head()

,id,title,contents
0,0,"[beyond, word, importance, :, using, contextua...","[the, driving, force, behind, the, recent, suc..."
1,1,"[generating, wikipedia, by, summarizing, long,...","[we, show, that, generating, english, wikipedi..."
2,2,"[mask, generative, adversarial, network, :, be...","[recurrent, neural, networks, are, a, common, ..."
3,3,"[natural, text, to, speach, synthesis, by, con...","[this, paper, describes, neural, network, mode..."
4,4,"[shaped, :, shared, private, encoder, decoder,...","[supervised, training, of, abstractive, langua..."



### Extract only word


In [42]:
import string

In [43]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [44]:
doc.filter(regex=r'[a-zA-Z]+')
doc['title'] = doc['title'].apply(lambda x: [a for a in x if a not in string.punctuation])
doc['contents'] = doc['contents'].apply(lambda x: [a for a in x if a not in string.punctuation])
doc.head()

,id,title,contents
0,0,"[beyond, word, importance, using, contextual, ...","[the, driving, force, behind, the, recent, suc..."
1,1,"[generating, wikipedia, by, summarizing, long,...","[we, show, that, generating, english, wikipedi..."
2,2,"[mask, generative, adversarial, network, bette...","[recurrent, neural, networks, are, a, common, ..."
3,3,"[natural, text, to, speach, synthesis, by, con...","[this, paper, describes, neural, network, mode..."
4,4,"[shaped, shared, private, encoder, decoder, fo...","[supervised, training, of, abstractive, langua..."


### lemmatizing


In [47]:
wnl = WordNetLemmatizer()

In [48]:
doc['title'] = doc['title'].apply(lambda x: [wnl.lemmatize(a) for a in x])
doc['contents'] = doc['contents'].apply(lambda x: [wnl.lemmatize(a) for a in x])
doc.head()

,id,title,contents
0,0,"[beyond, word, importance, using, contextual, ...","[the, driving, force, behind, the, recent, suc..."
1,1,"[generating, wikipedia, by, summarizing, long,...","[we, show, that, generating, english, wikipedi..."
2,2,"[mask, generative, adversarial, network, bette...","[recurrent, neural, network, are, a, common, m..."
3,3,"[natural, text, to, speach, synthesis, by, con...","[this, paper, describes, neural, network, mode..."
4,4,"[shaped, shared, private, encoder, decoder, fo...","[supervised, training, of, abstractive, langua..."


In [18]:
doc['title']

0    [beyond, word, importance, using, contextual, ...
1    [generating, wikipedia, by, summarizing, long,...
2    [mask, generative, adversarial, network, bette...
3    [natural, text, to, speach, synthesis, by, con...
4    [shaped, shared, private, encoder, decoder, fo...
5    [probabilistic, road, map, reinforcement, lear...
6    [collective, robot, reinforcement, learning, w...
7    [deep, reinforcement, learning, for, robotic, ...
8    [learning, to, manipulate, granular, media, wi...
9    [supervision, via, competition, robot, adversa...
Name: title, dtype: object

In [75]:
doc['all'] = doc['title'] + doc['contents']
doc['all'] = doc['all'].apply(lambda x: ' '.join(x))

In [76]:
doc.head()

,id,title,contents,all
0,0,"[beyond, word, importance, using, contextual, ...","[the, driving, force, behind, the, recent, suc...",beyond word importance using contextual decomp...
1,1,"[generating, wikipedia, by, summarizing, long,...","[we, show, that, generating, english, wikipedi...",generating wikipedia by summarizing long seque...
2,2,"[mask, generative, adversarial, network, bette...","[recurrent, neural, network, are, a, common, m...",mask generative adversarial network better tex...
3,3,"[natural, text, to, speach, synthesis, by, con...","[this, paper, describes, neural, network, mode...",natural text to speach synthesis by conditioni...
4,4,"[shaped, shared, private, encoder, decoder, fo...","[supervised, training, of, abstractive, langua...",shaped shared private encoder decoder for text...


In [77]:
doc.to_csv(path+'abstract_preprocessed.csv', index=False)